In [21]:
import math
import collections
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
plt.style.use ('fivethirtyeight')
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [2]:
data = pd.read_csv('NSE-TATAGLOBAL.csv')

In [15]:
df = pd.DataFrame(data)
df

,Date,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs)
0,2018-09-28,234.05,235.95,230.20,233.50,233.75,3069914,7162.35
1,2018-09-27,234.55,236.80,231.10,233.80,233.25,5082859,11859.95
2,2018-09-26,240.00,240.00,232.50,235.00,234.25,2240909,5248.60
3,2018-09-25,233.30,236.75,232.00,236.25,236.10,2349368,5503.90
4,2018-09-24,233.55,239.20,230.75,234.00,233.30,3423509,7999.55
...,...,...,...,...,...,...,...,...
2030,2010-07-27,117.60,119.50,112.00,118.80,118.65,586100,694.98
2031,2010-07-26,120.10,121.00,117.10,117.10,117.60,658440,780.01
2032,2010-07-23,121.80,121.95,120.25,120.35,120.65,281312,340.31
2033,2010-07-22,120.30,122.00,120.25,120.75,120.90,293312,355.17


In [16]:
df = df.drop(['Total Trade Quantity',"Turnover (Lacs)"],axis=1)
df

,Date,Open,High,Low,Last,Close
0,2018-09-28,234.05,235.95,230.20,233.50,233.75
1,2018-09-27,234.55,236.80,231.10,233.80,233.25
2,2018-09-26,240.00,240.00,232.50,235.00,234.25
3,2018-09-25,233.30,236.75,232.00,236.25,236.10
4,2018-09-24,233.55,239.20,230.75,234.00,233.30
...,...,...,...,...,...,...
2030,2010-07-27,117.60,119.50,112.00,118.80,118.65
2031,2010-07-26,120.10,121.00,117.10,117.10,117.60
2032,2010-07-23,121.80,121.95,120.25,120.35,120.65
2033,2010-07-22,120.30,122.00,120.25,120.75,120.90


In [17]:
df.isnull().sum()

Date     0
Open     0
High     0
Low      0
Last     0
Close    0
dtype: int64

In [18]:
df = df.set_index('Date')

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = df.index, y = df.Close,mode='lines',name='Close',marker_color = '#1F77B4'))

fig.update_layout(title='Closing price history of Tata Global',titlefont_size = 28,
              xaxis = dict(title='Date',titlefont_size=16,tickfont_size=14),height = 800,
              yaxis=dict(title='Price in INR (₹)',titlefont_size=16,tickfont_size=14),
              legend=dict(y=0,x=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'))
fig.show()

In [23]:
data = df.filter(['Close'])
dataset = data.values
training_data_len = math.ceil(len(dataset) * .8)
print("                                  ")
print("Length of the training data :",training_data_len)

                                  
Length of the training data : 1628


In [25]:
scaler = MinMaxScaler(feature_range=(0,1))
scaled_data = scaler.fit_transform(dataset)
print(scaled_data)

[[0.62418301]
 [0.62214052]
 [0.62622549]
 ...
 [0.1621732 ]
 [0.16319444]
 [0.16584967]]


In [26]:
train_data = scaled_data[0:training_data_len, :]
x_train = []
y_train = []
for i in range(60, len(train_data)):
    x_train.append(train_data[i-60:i, 0])
    y_train.append(train_data[i, 0])

In [28]:
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
print("Shape of training data :", x_train.shape)

Shape of training data : (1568, 60, 1)


In [50]:
model = Sequential()
model.add(LSTM(50, return_sequences = True, input_shape = (x_train.shape[1], 1)))
model.add(LSTM(50, return_sequences = True))
model.add(Dropout(0.2))
# model.add(LSTM(50, return_sequences = True))
# model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences = False))
model.add(Dense(25))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error')
model.fit(x_train, y_train, batch_size = 20, epochs = 50)

Epoch 1/50
79/79 [==============================] - 30s 153ms/step - loss: 0.0075
Epoch 2/50
79/79 [==============================] - 11s 143ms/step - loss: 0.0021
Epoch 3/50
79/79 [==============================] - 13s 163ms/step - loss: 0.0016
Epoch 4/50
79/79 [==============================] - 12s 150ms/step - loss: 0.0014
Epoch 5/50
79/79 [==============================] - 13s 168ms/step - loss: 0.0013
Epoch 6/50
79/79 [==============================] - 12s 152ms/step - loss: 0.0011
Epoch 7/50
79/79 [==============================] - 12s 149ms/step - loss: 9.9691e-04
Epoch 8/50
79/79 [==============================] - 12s 149ms/step - loss: 0.0010
Epoch 9/50
79/79 [==============================] - 13s 163ms/step - loss: 0.0011
Epoch 10/50
79/79 [==============================] - 12s 156ms/step - loss: 8.4898e-04
Epoch 11/50
79/79 [==============================] - 13s 160ms/step - loss: 8.1609e-04
Epoch 12/50
79/79 [==============================] - 13s 164ms/step - loss: 9.6507e-

In [51]:
test_data = scaled_data[training_data_len - 60: , :]
x_test = []
y_test = dataset[training_data_len:, :]

for i in range (60, len(test_data)):
    x_test.append(test_data[i - 60:i, 0])

x_test = np.array(x_test)
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [52]:
predictions = model.predict(x_test)
predictions = scaler.inverse_transform(predictions)

13/13 [==============================] - 4s 50ms/step


In [53]:
rsme = mean_squared_error(y_test, predictions,squared=False)
print("The RMSE value is : ",rsme)

The RMSE value is :  2.72240023603108


In [54]:
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions

<ipython-input-54-8f78faa8f9ca>:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [55]:
fig = go.Figure()
fig.add_trace(go.Scatter(x = train.index, y = train.Close,mode='lines',name='Close',marker_color = '#1F77B4'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Close,mode='lines',name='Val',marker_color = '#FF7F0E'))
fig.add_trace(go.Scatter(x = valid.index, y = valid.Predictions,mode='lines',name='Predictions',marker_color = '#2CA02C'))

fig.update_layout(title='Predicted plot of Tata Global',titlefont_size = 28,hovermode = 'x',
              xaxis = dict(title='Date',titlefont_size=16,tickfont_size=14),height = 800,
              yaxis=dict(title='Close price in INR (₹)',titlefont_size=16,tickfont_size=14),
              legend=dict(y=0,x=1.0,bgcolor='rgba(255, 255, 255, 0)',bordercolor='rgba(255, 255, 255, 0)'))

fig.show()

In [56]:
print("r2_score : ",r2_score(y_test,predictions))

r2_score :  0.9598559101716307
